## MGT-448 Final project code implementations

#### Author: Joachim Breitenstein
#### Date: 9/12/2024

## 3 Deep learning: Back-propagation

In [72]:
import numpy as np

# Define the sigmoid activation function
def sigmoid(x, derivative=False):
    f = 1/(1+np.exp(-x))
    return f*(1-f) if derivative else f

# Define the cost function
def cost_function(output, target, derivative=False): 
    E = np.mean((output - target) ** 2)
    return output - target if derivative else E

class NeuralNetwork:
    def __init__(self, layer_sizes): 
        
        #if no weights or biases are defined, initiate random values
        self.weights = [np.random.randn(y, x) for x, y in zip(layer_sizes[:-1], layer_sizes[1:])]
        self.biases = [np.random.randn(y, 1) for y in layer_sizes[1:]]

    def forward_pass(self, inputs):
        activation = np.array(inputs).reshape(-1, 1) # Convert inputs to 2D column vector if it is not already
        activations = [activation]  #append all activations for all layers
        int_activations = []  #store all outputs for all layers

        # Iterate over the weights and biases to calculate the activations for each layer
        for weight, bias in zip(self.weights, self.biases):
            z = np.dot(weight, activation) + bias
            int_activations.append(z)
            activation = sigmoid(z, derivative=False)
            activations.append(activation)

        return activations[-1], activations, int_activations
    
    def backpropagation(self, inputs, targets, learning_rate):
        targets = np.array(targets).reshape(-1, 1) # Convert targets to 2D column vector
        output, activations, zs = self.forward_pass(inputs)  # Forward pass
        delta = cost_function(output, targets, derivative=True) * sigmoid(zs[-1], derivative=True)  # Output layer error
        nabla_b = [np.zeros(b.shape) for b in self.biases]  # Gradient for biases
        nabla_w = [np.zeros(w.shape) for w in self.weights]  # Gradient for weights

        # Output layer
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].T)

        # Backpropagate through layers
        for l in range(2, len(self.weights) + 1):
            z = zs[-l]
            sp = sigmoid(z, derivative=True)
            delta = np.dot(self.weights[-l+1].T, delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].T)

        # Update weights and biases
        self.weights = [w - learning_rate * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - learning_rate * nb for b, nb in zip(self.biases, nabla_b)]

        cost = cost_function(output, targets, derivative=False)  # Compute cost for monitoring
        print("Cost after first feed-forward pass:", cost)
        return output, self.weights, self.biases



#Define architechture of network as list 
layer_sizes = [2, 2, 2] 

#initialize weights/biases as defined in the assignment
w1, w2, w3, w4, w5, w6, w7, w8 = 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45
b1, b2 = 0.3, 0.5

# Initialize network based on size specification in 'layer_sizes'
NN = NeuralNetwork(layer_sizes)
NN.weights = [
    np.array([[w1, w3], [w2, w4]]),  
    np.array([[w5, w7], [w6, w8]])   
]
NN.biases = [
    np.array([[b1], [b1]]),
    np.array([[b2], [b2]])  
]

#Perform forward pass with the given inputs
inputs = [0.05, 0.1]
output, activations, int_activations = NN.forward_pass(inputs)
print("Neural network ouput after one feed-forward:", output)

targets = [0.1, 0.9]
output = NN.backpropagation(inputs, targets, learning_rate=0.4) 

#Print the new weights and biases
print("Updated weights and biases after one backpropagation:")
for i, (w, b) in enumerate(zip(NN.weights, NN.biases)):
    print(f"Layer {i+1} weights:\n{w}")
    print(f"Layer {i+1} biases:\n{b}")

Neural network ouput after one feed-forward: [[0.71240836]
 [0.72417275]]
Cost after first feed-forward pass: 0.20297960811862323
Updated weights and biases after one backpropagation:
Layer 1 weights:
[[0.09987654 0.19975309]
 [0.14983274 0.24966549]]
Layer 1 biases:
[[0.29753085]
 [0.29665488]]
Layer 2 weights:
[[0.27086331 0.3707717 ]
 [0.35815567 0.45818131]]
Layer 2 biases:
[[0.44981125]
 [0.51404836]]


#### 3. Neural Network, answer:

From the first feed-forward we get the following output values: [0.71240836, 0.72417275]

Further, after one back-propagation, we get the following updated weights and biases: 

Input-Hidden layer weights: 0.09987654, 0.19975309, 0.14983274, 0.24966549
Input-Hidden layer biases: 0.29753085, 0.29665488

Hidden-Output layer weights: 0.27086331,0.3707717, 0.35815567, 0.45818131
Hidden-Output layer biases: 0.44981125, 0.51404836

 


## 4 Implementation of MCMC



In [60]:
import numpy as np
import scipy.io

# Load the data file from .mat format
data = scipy.io.loadmat('datamcmc.mat')
samples = data['X'].flatten()  # Extract the samples

# Define function for Gaussian probability density function 
def gaussian_pdf(x, mean, variance):
    return (1 / np.sqrt(2 * np.pi * variance)) * np.exp(-0.5 * ((x - mean) ** 2) / variance)

# Define function for likelihood function for the mixture of two Gaussians
def likelihood(samples, mu_f, mu_g, var_f, var_g):
    return np.product(0.5 * (gaussian_pdf(samples, mu_f, var_f) + gaussian_pdf(samples, mu_g, var_g)))

# Metropolis-Hastings algorithm for MCMC 
def metropolis_hastings(samples, initial_theta, iterations, burn_in):
    theta_chain = np.zeros((iterations, 2))
    theta_current = initial_theta
    for i in range(iterations):
        # estimate new theta
        theta_proposal = theta_current + np.random.normal(0, 0.5, 2)
        
        # Calculate likelihood ratio
        likelihood_current = likelihood(samples, theta_current[0], theta_current[1], var_f, var_g)
        likelihood_proposal = likelihood(samples, theta_proposal[0], theta_proposal[1], var_f, var_g)
        likelihood_ratio = likelihood_proposal / likelihood_current
        
        # Acceptance probability
        acceptance_probability = min(1, likelihood_ratio)
        if np.random.rand() < acceptance_probability:  # Accept or reject the proposal
            theta_current = theta_proposal
        
        theta_chain[i] = theta_current  # Store the current theta
    
    # Calculate the average of the last samples after burin-in for estimation of theta
    theta_estimate = np.mean(theta_chain[-burn_in:], axis=0)
    
    return theta_chain, theta_estimate

#initialize given parameters
theta_0 = np.array([3, -3])
var_f = 1
var_g = 4

# Run the MCMC algorithm for 2000 iterations. Only use last 1000 iterations for theta estimate
iterations = 2000
burn_in = 1000
theta_chain, theta_estimate = metropolis_hastings(samples, theta_0, iterations, burn_in)
print("Final theta estimate:", theta_estimate)


/var/folders/l7/59k2y53s6yg1fpd2vwhf5m2c0000gn/T/ipykernel_70058/681696774.py:25: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  likelihood_current = likelihood(samples, theta_current[0], theta_current[1], var_f, var_g)
/var/folders/l7/59k2y53s6yg1fpd2vwhf5m2c0000gn/T/ipykernel_70058/681696774.py:26: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  likelihood_proposal = likelihood(samples, theta_proposal[0], theta_proposal[1], var_f, var_g)
/var/folders/l7/59k2y53s6yg1fpd2vwhf5m2c0000gn/T/ipykernel_70058/681696774.py:27: RuntimeWarning: invalid value encountered in scalar divide
  likelihood_ratio = likelihood_proposal / likelihood_current


Final theta estimate: [-0.8429537   1.54496989]


#### 4. MCMC, answer:
As such, we find the following final $\theta$ as the average over the last 1000 $\theta$ values: 

$\theta = [\mu_f, \mu_g]^T = [-0.8429537, 1.54496989]$ 

## 5 Chow-Liu

In [75]:
import numpy as np
from scipy.sparse.csgraph import minimum_spanning_tree

# Implementing mututal(s) function to estimate mutual information 
# between information between two Normal random variables
def Mutual(S): 
    sigma_11, sigma_22 = S[0, 0], S[1, 1]
    sigma_12 = S[0, 1]
    return -0.5 * np.log(1 - (sigma_12 ** 2) / (sigma_11 * sigma_22))

def check_MI(samples):
    cov_matrix = np.cov(samples, rowvar=False)
    return Mutual(cov_matrix)

# Define the Chow-Liu algorithm
def ChowLiu(samples):
    num_variables = samples.shape[1]
    # Initialize the matrix for mutual information values
    mutual_info_matrix = np.zeros((num_variables, num_variables))

    # Calculate mutual information for every pair of variables
    for i in range(num_variables):
        for j in range(i+1, num_variables):
            mutual_info_matrix[i, j] = Mutual(samples[:, [i, j]])

    # Use the negative mutual information matrix to find the maximum spanning tree
    mst = minimum_spanning_tree(-mutual_info_matrix).toarray()
    
    # Convert the minimum spanning tree to an adjacency matrix
    adj_matrix = np.where(mst != 0, 1, 0)
    return adj_matrix

cov = np.array([[1, 0.3], [0.3, 2]])  # define given covariance matrix
mean = np.zeros(2)
true_mutual_info = Mutual(cov) 
np.random.seed(42) #initialize random seed for reproducability
n_samples = [1000, 10000] #define sample sizes

for n in n_samples:
    # Generate samples using np.random.multivariate using covariance matrix and mean from ditribution
    samples = np.random.multivariate_normal(mean, cov, size=n)
    # Estimate mutual information from the generated samples
    estimated_mi = check_MI(samples)
    # Calculate the estimation error
    error = np.abs(estimated_mi - true_mutual_info)
    adjacency_matrix = ChowLiu(samples) #computes adjencency matrix for input pxp matrix
    print(f"Error using {n} samples:", error)
    print(f"Adjacency_matrix of final tree using {n} samples:", adjacency_matrix)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Error using 1000 samples: 0.0053935106017962385
Adjacency_matrix of final tree using 1000 samples: [[0 0]
 [0 0]]
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Error using 10000 samples: 0.004184659949877576
Adjacency_matrix of final tree using 10000 samples: [[0 0]
 [0 0]]


/var/folders/l7/59k2y53s6yg1fpd2vwhf5m2c0000gn/T/ipykernel_70058/1827234622.py:9: RuntimeWarning: invalid value encountered in log
  return -0.5 * np.log(1 - (sigma_12 ** 2) / (sigma_11 * sigma_22))


#### 5. Chow-Liu, answer
From running the Chow-Liu algorithm for 1000 and 10000 iterations we get the following erros: 

- For 1000 iteratisons: 0.0053935106017962385
- For 10000 iterations: 0.004184659949877576

As such, it is clear that the error goes down when the number of iterations is increased. 

## 6 Structure learning with SGS Algorithm

In [76]:
import numpy as np
from scipy.stats import norm
from numpy.linalg import inv
from typing import List
from itertools import combinations


def CItest(D: np.ndarray, X: int, Y: int, Z: List[int]) -> int:
    """
    Test for conditional independence between variables X and Y given a set Z in dataset D.

    Parameters:
    D (numpy.ndarray): A matrix of data with size n*p, where n is the number of samples and p is the number of variables.
    X (int): Index of the first variable (zero-based indexing).
    Y (int): Index of the second variable (zero-based indexing).
    Z (List[int]): A list of indices for variables in the conditioning set (zero-based indexing).

    Returns:
    int: 1 if conditionally independent, 0 otherwise.
    """
    
    # Significance level
    alpha = 0.06

    # Number of samples
    n = D.shape[0]

    # Select columns corresponding to X, Y, and Z from the dataset
    DD = D[:, [X, Y] + Z]
    
    # Compute the precision matrix
    R = np.corrcoef(DD, rowvar=False)
    P = inv(R)

    # Calculate the partial correlation coefficient and Fisher Z-transform
    ro = -P[0, 1] / np.sqrt(P[0, 0] * P[1, 1])
    zro = 0.5 * np.log((1 + ro) / (1 - ro))

    # Test for conditional independence
    c = norm.ppf(1 - alpha / 2)
    if abs(zro) < c / np.sqrt(n - len(Z) - 3):
        CI = 1
    else:
        CI = 0

    return CI

def sgs(data):
    n_variables = data.shape[1]
    skeleton = np.ones((n_variables, n_variables), dtype=int) - np.eye(n_variables, dtype=int) #initialize skeleton with 1's. 

    for x, y in combinations(range(n_variables), 2):
        # this loop checks conditional independence for all subsets Z of the remaining variables
        for z_size in range(n_variables - 2):
            for z in combinations(set(range(n_variables)) - {x, y}, z_size):
                if CItest(data, x, y, list(z)):
                    skeleton[x, y] = skeleton[y, x] = 0  # Remove edge X-Y from the skeleton
                    break  # Break out of the loop if a set Z is found that makes X and Y conditionally independent

    return skeleton 

#test algorithm for all three datasets
for i in range(3): 
    D = np.load(f"Q6_files/D{i+1}.npy")
    skeleton_adj_matrix = sgs(D)
    print(f"Skeleton adjencency matrix for datafile D{i+1}:", skeleton_adj_matrix)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
